In [ ]:
# %load full_pipeline.py
## %load full_pipeline.py

#%load_ext line_profiler

# Step1_computeFeaturesFromRawData.m
import itertools
import os
import numpy as np
import hdf5storage
import spectrum
from joblib import Parallel
from joblib import delayed
from numpy.core.defchararray import lower
from numpy import floor
import swiftclient
import pandas as pd
#from compute_spectrogram_sunhaoqi import mtspecgram_shq
from get_features import get_features
from scipy.signal import butter, filtfilt, lfilter
from runtime import cellarray, matlabarray, cat, cell, arange, size, disp, find, dot, copy, length, Dict2Obj,catstr

from mtspec import mt_spectrogram


def load(fp):
    matfile = hdf5storage.loadmat(fp)
    
    return matfile


# old matlab ITF
def save(fp, d):
    fp = fp + '.mat'

    if os.path.exists(fp):
        os.remove(fp)
    for e in d.keys():
        obj = d[e]
        print('key', e, 'shape', obj.shape, obj.dtype)
        print(obj)

        def apply_f(a, f):
            # if isinstance(a, list):
            print(str(type(a)))
            if 'cellarray' in str(type(a)):
                print('cell')
                return map(lambda t: apply_f(t, f), a)
            if 'ndarray' in str(type(a)):
                print('ndarray')
                return a
            else:
                print('other')
                return f(a)

        out_obj = list(apply_f(cellarray(obj), lambda e: e.copy()))

        hdf5storage.write(data=out_obj, path='/%s' % e, filename=fp,
                          store_python_metadata=False, matlab_compatible=True)

import scipy
class load_mat():
    def __init__(self, filename, mode='r'):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        print('open')
        # self.open_file = open(self.filename, self.mode)
        return 0

    def __exit__(self, *args):
        # self.open_file.close()
        print('close')


def Step1_computeFeaturesFromRawData(fileName0=None,*args,**kwargs):
    ##### 1.1 Compute spectrograms ####
    objStorCred = {
      "auth_url": "https://identity.open.softlayer.com",
      "project": "object_storage_20ff227d_d66c_495a_a316_cd99e80a9e6f",
      "projectId": "a9fb4d478e3d40a8bbd54c5a2ecf25a3",
      "region": "dallas",
      "userId": "6a4cc8251c1940179a6cccc9098a15e0",
      "username": "admin_fd35793e7cf915d9a5a9c768b068029cf6d720b9",
      "password": "kDTcKA2H(3eo5.G0",
      "domainId": "a350f0fe7fb44571b29305706a12c95a",
      "domainName": "1334933",
      "role": "admin"
    }
    conn = swiftclient.Connection(auth_version='3',key=objStorCred['password'],
                              authurl=objStorCred['auth_url']+'/v3',
                             os_options={"project_id":objStorCred['projectId'],
                                       "user_id":objStorCred['userId'],
                                       "region_name":objStorCred['region']})
    #if not os.path.isfile(os.path.basename(fileName0)):
    
    absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
    fileName0=absdir+fileName0
    
    #if not os.path.isfile(fileName0):
    #    print('writing to local',fileName0)
   # 
   #     container = 'MGH'
   #     resp_headers, obj_contents = conn.get_object(container, fileName0)
   #     #with open(os.path.basename(fileName0), 'wb') as local:
   #     with open(fileName0, 'wb') as local:
   #         local.write(obj_contents)

    #if (1):
    try:    
        
        matfile = hdf5storage.loadmat(fileName0)

        Fs = cellarray(int(np.round(matfile['Fs'])))
        start_time = cellarray(matfile['start_time'])
        channels = cellarray(matfile['channels'])
        data = cellarray(matfile['data'])
        # print(size(data))
        #     data=cellarray(data[:,0:100])

        # New: Resample to 200 Hz!!! %
        Fs = Fs.item()
        P = 200
        Q = int(Fs)
        data = scipy.signal.resample(data.T, int(len(data.T) * P / Q))
        ## data = np.resample(data, 200, Fs)
        data = cellarray(data.T)
        Fs = 200

        def strrep_s(a, x, y):
            return a.replace(x, y)

        # =strcmp(labels,"['f8']")
        strrep = np.vectorize(strrep_s)

        print('channels', channels)
        print(size(channels, 1))
        # df_s0=pd.DataFrame()
        labels = copy(channels)
        # print(labels.shape)

        for i in arange(1, size(channels, 1)).reshape(-1):
            x = strrep(channels[i, :], ' ', '')
            labels[i] = lower(x)

        disp('load 0')
        disp('compute 1','calculate locational differential voltage')
        # pd=copy(pwd)
        movingwin = matlabarray(cat(2, 2))

        # dictobj
        params = Dict2Obj({'default': 1})

        params.pad = copy(0)
        params.fpass = copy(cat(0.5, 20))
        params.err = copy(0)
        params.trialave = copy(0)
        params.tapers = copy(cat(2, 3))
        params.Fs = copy(200)

        # spect=cell(4,4)
        # COI=numpy.empty((4, ), dtype=object)
        COI = cell(4, )
        COI[1] = cellarray([['fp1', 'f7'], ['f7', 't3'], ['t3', 't5'], ['t5', 'o1']])
        COI[2] = cellarray([['fp1', 'f3'], ['f3', 'c3'], ['c3', 'p3'], ['p3', 'o1']])
        COI[3] = cellarray([['fp2', 'f4'], ['f4', 'c4'], ['c4', 'p4'], ['p4', 'o2']])
        COI[4] = cellarray([['fp2', 'f8'], ['f8', 't4'], ['t4', 't6'], ['t6', 'o2']])

        # test
        def strcmp_s(x, y):
            return x == y

        strcmp = np.vectorize(strcmp_s)

        dataB = cell(4, 4)

        # print(data.shape,data.__class__,dataB.shape,dataB.__class__,data.dtype,dataB.dtype)
        for kk in arange(1, 4).reshape(-1):
            # print('kk',kk)
            coi = COI[kk]
            print('coi', coi, size(coi, 1), size(coi, 2), size(coi))

            for k in arange(1, size(coi, 1)).reshape(-1):
                print('kk', kk, 'k', k, coi[k, 1], coi[k, 2])
                # print(coi[k, 1 - 1], coi[k, 2 - 1])
                # print('k',k,coi,'labels')
                # print(coi[k,1-1],coi[k,2-1])

                c1 = find(strcmp(labels, coi[k, 1]))
                c2 = find(strcmp(labels, coi[k, 2]))
                c1 = c1.item()
                c2 = c2.item()
                # dataB[kk,k]=data(c1,arange()) - data(c2,arange())
                # print('items',c1,c2)
                dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())

        d = {}
        d['dataB'] = dataB
        # outp=catstr(targetFolder, '/strcmp_', fileName0)
        # print(outp)
        # save(outp, d)


        print('compute 2', 'compute spectrogram for each channel')
        print('params', params.Fs, movingwin)
        # for kk,k in product([1,2,3,4],[1,2,3,4]):
        #     print(kk,k)
        #     S, stimes, sfreqs = mt_spectrogram(dataB[kk,k], Fs)

        # Par version
        n_jobs = -1  # number of cpus for parallel computing, -1 is all cpus
        verbose = True  # verbosity in parallel computing
        res = Parallel(n_jobs=n_jobs, verbose=verbose)(
            delayed(mt_spectrogram)(dataB[kk,k], Fs) for kk,k in itertools.product([1,2,3,4],[
                1,2,3,4])
        )

        spect = cell(4, 4)
        print(len(res))

        for i, rr in enumerate(res):
            # rr[0][freq_good_ids]
            ii=i % 4
            iii=floor(i/4)
            spect[iii, ii + 1] = rr[0].T

        stimes=res[0][1]
        sfreqs = res[0][2]
        
                
        d = {}
        d['dataB'] = dataB
        # outp=catstr(targetFolder, '/compute2_', fileName0)
        # save(outp, d)

        # compute regional averages
        disp('compute 3', 'compute regional averages')
        # R=cell(4,1)
        R = cell(4, 1)
        # R=np.empty((4,1),dtype=object)
        for kk in arange(1, 4).reshape(-1):
            # print('kk',kk,'size',size(spect[kk,1-1]),spect[kk,1-1].shape,tuple(size(spect[kk,1-1])))
            T = np.zeros(size(spect[kk, 1]))
            # print('T',T)
            # T = zeros(sizespect[kk, 1 - 1].shape[0])
            print('T', T.shape)
            for k in arange(1, 4).reshape(-1):
                # print('kk',kk,'k',k)

                T = T + spect[kk, k]
            R[kk] = T / 4

        print('save 0')
        # save(cat(targetFolder,'/spect_',fileName0),'sfreqs','stimes','R','dataB','Fs','-v7.3')
        d = {}
        d['dataB'] = dataB
        d['sfreqs'] = sfreqs
        d['stimes'] = stimes
        # d['sfreqs']=sfreqs2
        # d['stimes']=stimes2

        d['Fs'] = cellarray(Fs)
        d['R'] = R
        # outp=catstr(targetFolder, '/spect_', fileName0)
        # save(outp, d) # 7.3 by default


        #### 1.2 Compute features ####
        disp('compute 4', 'compute features')

        import multiprocessing
        print('CPU',multiprocessing.cpu_count())

        epoch_length = dot(Fs, 2)
        window_length = dot(cat(14, 10, 6, 2), Fs)

        # filtering of data
        fnyq = Fs / 2  # nyquist frequency
        fc_high = 50
        fc_low = 0.5
        #  cutoff frequencies
        # b,a=butter(6,cat(fc_low / fnyq,fc_high / fnyq),'bandpass',nargout=2)
        b, a = butter(6, cat(fc_low / fnyq, fc_high / fnyq), 'bandpass')

        for kk in arange(1, 4).reshape(-1):
            for k in arange(1, 4).reshape(-1):
                # print('kk',kk,'k',k)
                temp = dataB[kk, k]
                # dataB[kk, k] = filtfilt(b, a, temp)
                # dataB[kk,k]=filtfilt(b,a,temp,padtype='odd')

                # dataB[kk, k] = lfilter(b, a, temp)
                dataB[kk, k] = filtfilt(b, a, temp, padtype='odd', padlen=3 * (np.amax((len(a), len(b))) - 1))

        # disp('save 1', outp)
        # outp=catstr(targetFolder, '/filtered_data_', fileName0)
        d = {}
        d['dataB'] = dataB
        d['Fs'] = cellarray(Fs)
        # save(outp, dataB, Fs)
        # save(outp, d)

        disp('compute 5','calc features')
        spectwindow_length = floor(window_length / Fs / 2)

        # fix: per jingjing
        # nr_epochs=floor(length(dataB[1,1](arange(window_length[1] - epoch_length,end()))) / epoch_length)
        # N=length(dataB[1,1])
        # N = length(dataB[1-1, 1-1])
        # nr_epochs=floor((N - (window_length[1] - epoch_length) - Fs) / epoch_length)
        # nr_features=144

        N = length(dataB[1, 1])
        nr_epochs = floor((N - (window_length[1] - epoch_length) - Fs) / epoch_length)
        nr_features = 144

        features = cell(4, nr_features, nr_epochs)
        for kk in arange(1, 4).reshape(-1):
            # for kk in arange(1,1).reshape(-1):
            #         disp(kk);
            # spect=R[kk,1]
            spect = R[kk, 1]

            data = dataB[kk, :]
            # print('kk',kk)
            # for i in arange(1,nr_epochs).reshape(-1):
            # # TEST features
            # # print('kk',kk,'i',i)
            # # features[kk,:,i]=get_features(data,i,window_length,spect,sfreqs,spectwindow_length,Fs)
            #
            #     features[kk, :, i] = get_features(data, int(i.item()), window_length, spect, sfreqs, spectwindow_length, Fs)


            #Par version
            n_jobs = -1 # number of cpus for parallel computing, -1 is all cpus
            verbose = True  # verbosity in parallel computing
            res = Parallel(n_jobs=n_jobs, verbose=verbose)(
                # delayed(compute_spec_each_seg)(eeg[window_start[wi]:window_start[wi] + window_length, :], NW, Fs) for wi in range(window_num)
                delayed(get_features)(data, int(i.item()), window_length, spect, sfreqs, spectwindow_length, Fs) for i
                in arange(1, nr_epochs).reshape(-1)
            )

            print(len(res))
            for i, rr in enumerate(res):
                # rr[0][freq_good_ids]
                features[kk, :, i + 1] = rr[:]

        # features=reshape(features,cat(dot(4,nr_features),length(features)))
        features = features.reshape(cat(dot(4, nr_features), length(features)))
        features = features.T

        # outp=catstr(targetFolder, '/features_', fileName0)
        disp('save 2')
        # save(cat(targetFolder,'/features_',fileName0),'features','window_length')
        # d={}
        # d['features']=cellarray(features)
        # d['window_length']=cellarray(window_length)
        # save(outp, features, window_length)
        # save(outp, d)

        # save the df locally
        saveFeats = pd.DataFrame(features.T)
        print(saveFeats)
        try:
            print(fileName0)
            onlyFiName = os.path.basename(fileName0)
            fp = os.path.basename(fileName0) + '.csv'
            print(fp)
            savep = abs_dir + 'MGH-100P/features_' + fp
            print('save to ', savep)
            if not os.path.isfile(savep):
                saveFeats.to_csv(savep)

        except:
            print('[ERROR]', 'saving to csv')

        # timedf
        ######features (numpy array of arrays)
        # transpose features
        saveFeats = features.T

        # turn this into a pandas df
        saveFeats = pd.DataFrame(saveFeats)

        # add time stamp
        saveFeats.reset_index(inplace=True)
        fileNum = fileName0[:len(fileName0) - 4]
        import re
        fileNum = re.search('(\d+)$', fileNum).group(0)
        sampleRate = round(Fs)

        def timeStamp(ind, fileNum, sampleRate):
            start = (int(fileNum) - 1) * 99999
            row = start + int(ind)
            time = row * int(sampleRate)
            return time

        saveFeats['time'] = saveFeats['index'].apply(lambda e: timeStamp(e, fileNum, sampleRate))

        # add case id
        caseNum = int(re.search(r'\d+', fileName0).group())
        saveFeats['case'] = caseNum

        try:
            print(fileName0)
            onlyFiName = os.path.basename(fileName0)
            fp = os.path.basename(fileName0) + '.csv'
            print(fp)
            savep = absdir + 'MGH-100P/new_features_' + fp
            print('save to ', savep)
            if not os.path.isfile(savep):
                saveFeats.to_csv(savep)

        except:
            print('[ERROR]', 'saving to csv')

        # copy the local csv to obj storage
        # print("Case"+str(caseNum)+"/"+onlyFiName+".csv")
        try:
            with open(onlyFiName + ".csv", 'r') as local:
                # conn.put_object(container, "Case"+str(caseNum)+"/"+onlyFiName+".csv",contents=local,content_type='text/plain')
                conn.put_object(container, onlyFiName + ".csv", contents=local, content_type='text/plain')
        except:
            print('[ERROR]', 'saveint to object store')

        # delete the local copy
        # os.remove(onlyFiName+".csv")
        print('saving', onlyFiName + ".csv")

        status = saveFeats.shape
    except Exception as e:
        status = str(e)
    
    return (fileName0, status,start_time,Fs,data.shape)
    #return (fileName0, status)
    
    
# test
runt=False
if (runt):
    dataPath='./'
    targetFolder='./Output'
    #absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
    #filep = absdir+"Case4/Case4_seg2.mat"
    #filep = "test2.mat"
    casep='Case89/Case89_seg3.mat'
    filep = casep
    #print(os.path.basename(filep)+'.csv')
    fileName0=filep
    #print(fileName0)
    abs_dir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
    onlyFiName = os.path.basename(fileName0)
    fp=os.path.basename(fileName0)+'.csv'
    #print(fp)
    savep=abs_dir+'MGH-100P/features_'+fp
    #print('save to ',savep)
    
    
    import datetime
    print(np.__version__)
    startdt=datetime.datetime.now()
    try:
        1
        #%lprun -f Step1_computeFeaturesFromRawData(fileName0=filep)
        Step1_computeFeaturesFromRawData(fileName0=filep)
    except Exception as e:
        print('[ERROR]','error occured',str(e))
    enddt=datetime.datetime.now()
    
    print('finish',enddt - startdt)

In [ ]:
#!pip install --user --upgrade python-swiftclient

In [36]:
#!pip install numba

In [1]:
!ls MGH-100P
#!rm 

features_Case1_seg12.mat.csv	    new_features_Case17_seg3.mat.csv
new_features_Case101_seg1.mat.csv   new_features_Case17_seg7.mat.csv
new_features_Case101_seg2.mat.csv   new_features_Case18_seg4.mat.csv
new_features_Case101_seg6.mat.csv   new_features_Case1_seg12.mat.csv
new_features_Case102_seg1.mat.csv   new_features_Case2_seg1.mat.csv
new_features_Case102_seg4.mat.csv   new_features_Case31_seg2.mat.csv
new_features_Case102_seg6.mat.csv   new_features_Case33_seg5.mat.csv
new_features_Case103_seg6.mat.csv   new_features_Case35_seg8.mat.csv
new_features_Case103_seg9.mat.csv   new_features_Case36_seg6.mat.csv
new_features_Case104_seg2.mat.csv   new_features_Case38_seg4.mat.csv
new_features_Case105_seg2.mat.csv   new_features_Case39_seg4.mat.csv
new_features_Case105_seg5.mat.csv   new_features_Case43_seg2.mat.csv
new_features_Case106_seg1.mat.csv   new_features_Case44_seg3.mat.csv
new_features_Case106_seg3.mat.csv   new_features_Case45_seg7.mat.csv
new_features_Case107_seg1.mat.csv   ne

In [10]:
#!pip install line_profiler
#!rm full_pipeline.py
#!rm get_features.py
!ls *.py

compute_spectrogram_sunhaoqi.py  full_pipeline.py  mtspec.py
fcn_shannon_entro.py		 get_features.py   runtime.py


In [1]:
%time teraFile = spark.read.text("TeraGen-1TB/")
%time teraFile.groupBy("value").count().take(100)
print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()

#teraFile.groupBy("value").count().take(100)

CPU times: user 1.17 ms, sys: 1.52 ms, total: 2.69 ms
Wall time: 2.44 s


KeyboardInterrupt: 

In [2]:
sc.addPyFile("fcn_shannon_entro.py")
sc.addPyFile("full_pipeline.py")
sc.addPyFile("compute_spectrogram_sunhaoqi.py")
sc.addPyFile("get_features.py")
sc.addPyFile("runtime.py")
sc.addPyFile("mtspec.py")

In [10]:
# Same bench on full script
import itertools
import os
import numpy as np
import hdf5storage
import spectrum
from joblib import Parallel
from joblib import delayed
from numpy.core.defchararray import lower
from numpy import floor
import swiftclient
import pandas as pd
#from compute_spectrogram_sunhaoqi import mtspecgram_shq
from get_features import get_features
from scipy.signal import butter, filtfilt, lfilter
from runtime import cellarray, matlabarray, cat, cell, arange, size, disp, find, dot, copy, length, Dict2Obj,catstr
import scipy
import multiprocessing as mp

from mtspec import mt_spectrogram

#@contextlib.contextmanager
class Timer(object):
    def __init__(self, verbose=False):
        self.verbose = verbose

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start
        self.msecs = self.secs * 1000  # millisecs
        if self.verbose:
            print('elapsed time: %f ms' % self.msecs)

import datetime
def testFunc(elt):
 #   with Timer() as t:
        startdt = datetime.datetime.now()
        meas_t=0
        try:
            #d=dict(sc._conf.getAll())
            #print(d['spark.executor.id'])

            absdir='/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/'
            elt=absdir+elt
            datasize=os.path.getsize(elt)
            matfile = hdf5storage.loadmat(elt)


            Fs = cellarray(int(np.round(matfile['Fs'])))
            start_time = cellarray(matfile['start_time'])
            channels = cellarray(matfile['channels'])
            data = cellarray(matfile['data'])
            
            cpu_count=mp.cpu_count()

            
            Fs = 200

            def strrep_s(a, x, y):
                return a.replace(x, y)

            strrep = np.vectorize(strrep_s)

            print('channels', channels)
            print(size(channels, 1))
            labels = copy(channels)

            for i in arange(1, size(channels, 1)).reshape(-1):
                x = strrep(channels[i, :], ' ', '')
                labels[i] = lower(x)

            disp('load 0')
            disp('compute 1','calculate locational differential voltage')
            movingwin = matlabarray(cat(2, 2))

            # dictobj
            params = Dict2Obj({'default': 1})

            params.pad = copy(0)
            params.fpass = copy(cat(0.5, 20))
            params.err = copy(0)
            params.trialave = copy(0)
            params.tapers = copy(cat(2, 3))
            params.Fs = copy(200)

            # spect=cell(4,4)
            # COI=numpy.empty((4, ), dtype=object)
            COI = cell(4, )
            COI[1] = cellarray([['fp1', 'f7'], ['f7', 't3'], ['t3', 't5'], ['t5', 'o1']])
            COI[2] = cellarray([['fp1', 'f3'], ['f3', 'c3'], ['c3', 'p3'], ['p3', 'o1']])
            COI[3] = cellarray([['fp2', 'f4'], ['f4', 'c4'], ['c4', 'p4'], ['p4', 'o2']])
            COI[4] = cellarray([['fp2', 'f8'], ['f8', 't4'], ['t4', 't6'], ['t6', 'o2']])

            # test
            def strcmp_s(x, y):
                return x == y

            strcmp = np.vectorize(strcmp_s)

            dataB = cell(4, 4)

            # print(data.shape,data.__class__,dataB.shape,dataB.__class__,data.dtype,dataB.dtype)
            for kk in arange(1, 4).reshape(-1):
                # print('kk',kk)
                coi = COI[kk]
                print('coi', coi, size(coi, 1), size(coi, 2), size(coi))

                for k in arange(1, size(coi, 1)).reshape(-1):
                    c1 = find(strcmp(labels, coi[k, 1]))
                    c2 = find(strcmp(labels, coi[k, 2]))
                    c1 = c1.item()
                    c2 = c2.item()

                    dataB[kk, k] = np.array((data[c1, :] - data[c2, :]).tolist())    


            # Step 2 - spectrogram
            n_jobs = -1  # number of cpus for parallel computing, -1 is all cpus
            verbose = True  # verbosity in parallel computing
            res = Parallel(n_jobs=n_jobs, verbose=verbose)(
                delayed(mt_spectrogram)(dataB[kk,k], Fs) for kk,k in itertools.product([1,2,3,4],[
                    1,2,3,4])
            )


            # Step 3
            spect = cell(4, 4)
            print(len(res))

            for i, rr in enumerate(res):
                # rr[0][freq_good_ids]
                ii=i % 4
                iii=floor(i/4)
                spect[iii, ii + 1] = rr[0].T

            stimes=res[0][1]
            sfreqs = res[0][2]
            
            
            
            
            
            msg=spect.shape
        except Exception as e:
            print('error')
            start_time=''
            data=np.array((0,0))
            datasize=0
            
            cpu_count=0
            msg=str(e)
    
    #meas_t=t
    #meas_t=0
        enddt = datetime.datetime.now()
        dur=enddt-startdt
        return (elt,start_time,startdt,enddt,meas_t,datasize,data.shape,cpu_count,dur,msg)

# Benching 8
runt=True
if (runt):
    #lfiles=!ls Case10*/*.mat
    lfiles=!find Case*/*.mat -size -100000
    
    print('num exec',sc._jsc.sc().getExecutorMemoryStatus().size())
    n_exec=sc._jsc.sc().getExecutorMemoryStatus().size()
    
    lsub=['Case1/Case1_seg12.mat','Case109/Case109_seg1.mat','Case89/Case89_seg3.mat']
    lsub=['Case1/Case1_seg10.mat','Case1/Case1_seg11.mat','Case17/Case17_seg7.mat','Case17/Case17_seg3.mat',\
          'Case11/Case11_seg4.mat','Case11/Case11_seg4.mat','Case1/Case1_seg12.mat','Case109/Case109_seg1.mat','Case89/Case89_seg3.mat']
    
    #lsub=['Case1/Case1_seg12.mat','Case1/Case1_seg12.mat','Case1/Case1_seg12.mat','Case1/Case1_seg12.mat',\
    #        'Case1/Case1_seg12.mat','Case1/Case1_seg12.mat','Case1/Case1_seg12.mat','Case1/Case1_seg12.mat']
    #lsub=['Case109/Case109_seg1.mat']
    lsub=lfiles[:n_exec]
    
    print('num data',len(lsub))
    
    distData = sc.parallelize(lsub)
    dtd=distData.repartition(n_exec)
    distData.cache()
    dtd.cache()
    
    print('num part',dtd.getNumPartitions())
    
    #testdtd=dtd.map(lambda e: testFunc(e))
    #testdtd.cache()
    #%time lres=testdtd.take(len(lsub))    
    #print(dtd.collect(),lres,len(lres)) 
    

num exec 53
num data 53
num part 53


In [11]:
dtd.count()

53

In [12]:
import datetime
print(datetime.datetime.now())

#testdtd=dtd.map(lambda e: mockFunc(e))
testdtd=dtd.map(lambda e: testFunc(e))
#testdtd.cache()
#%time lres=testdtd.take(len(lsub))
%time lres=testdtd.collect()
#%time lsrc=dtd.collect()
#print(lsrc,len(lsrc))
print(lres,len(lres)) 

print(datetime.datetime.now())

CPU times: user 78.6 ms, sys: 28.9 ms, total: 107 ms
Wall time: 5min 6s
[('/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/Case124/Case124_seg4.mat', cellarray([['09-29-2012 09:54:49']], dtype=object), datetime.datetime(2017, 7, 19, 14, 14, 59, 8134), datetime.datetime(2017, 7, 19, 14, 15, 19, 570636), 0, 30154738, (50, 679042), 48, datetime.timedelta(0, 20, 562502), (4, 4)), ('/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/Case112/Case112_seg5.mat', cellarray([['06-10-2012 11:17:05']], dtype=object), datetime.datetime(2017, 7, 19, 14, 14, 59, 473637), datetime.datetime(2017, 7, 19, 14, 15, 17, 401641), 0, 26923796, (50, 678541), 48, datetime.timedelta(0, 17, 928004), (4, 4)), ('/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/Case17/Case17_seg7.mat', cellarray([['08-07-2012 14:59:57']], dtype=object), datetime.datetime(2017, 7, 19, 14, 14, 57, 376172), datetime.datetime(2017, 7, 19, 14, 15, 12, 749348), 0, 29578255, (50, 597759), 48, dateti

In [20]:
import numpy as np
output = np.array(lres)
for line in output[np.argsort(output[:,2])]:
    print(line[2],line[3],line[8])

2017-07-19 14:14:55.785349 2017-07-19 14:15:11.217694 0:00:15.432345
2017-07-19 14:14:57.256871 2017-07-19 14:15:06.299287 0:00:09.042416
2017-07-19 14:14:57.365244 2017-07-19 14:15:11.053179 0:00:13.687935
2017-07-19 14:14:57.369022 2017-07-19 14:15:06.070349 0:00:08.701327
2017-07-19 14:14:57.376172 2017-07-19 14:15:12.749348 0:00:15.373176
2017-07-19 14:14:58.323404 2017-07-19 14:15:13.940994 0:00:15.617590
2017-07-19 14:14:58.326936 2017-07-19 14:15:36.788623 0:00:38.461687
2017-07-19 14:14:58.345580 2017-07-19 14:15:21.451646 0:00:23.106066
2017-07-19 14:14:58.409237 2017-07-19 14:16:39.507543 0:01:41.098306
2017-07-19 14:14:58.420911 2017-07-19 14:15:21.214175 0:00:22.793264
2017-07-19 14:14:58.422968 2017-07-19 14:15:09.820937 0:00:11.397969
2017-07-19 14:14:58.425545 2017-07-19 14:15:14.353527 0:00:15.927982
2017-07-19 14:14:58.426487 2017-07-19 14:15:19.913910 0:00:21.487423
2017-07-19 14:14:59.008134 2017-07-19 14:15:19.570636 0:00:20.562502
2017-07-19 14:14:59.165777 2017-07

In [ ]:
# Benching single - Case5_seg6.mat
import numpy as np
from full_pipeline import Step1_computeFeaturesFromRawData
#from compute_spectrogram_sunhaoqi import mtspecgram_shq, compute_spec_each_seg
#import datetime

import datetime
print(np.__version__)
startdt=datetime.datetime.now()
try:
    1
    
    justData3 = distData.flatMap(Step1_computeFeaturesFromRawData)
    justData3.cache()
    print(justData3)
    
    %time justData3.take(len(data))
    
except Exception as e:
    print('[ERROR]','error occured',str(e))
enddt=datetime.datetime.now()

print('finish',enddt - startdt)



1.11.0
PythonRDD[6] at RDD at PythonRDD.scala:48
CPU times: user 663 ms, sys: 434 ms, total: 1.1 s
Wall time: 2h 4min 53s
finish 2:04:53.885096


In [ ]:
!echo $HOME

In [ ]:
!ls $SPARK_CONFIG_HOME

In [ ]:
import os  
os.listdir(os.environ['SPARK_CONFIG_HOME'])

In [ ]:
def readSparkConfig(spark):  
    """
    A function to read the Spark Config of DSX
    :param spark: a string specifying which `spark-defaults.conf` to read. 
    :return: a dict, where the keys corespond to the parameters of `spark-defaults.conf` and the values are the values.
    """
    if not spark in os.listdir(os.environ['SPARK_CONFIG_HOME']):
        #print "'%s' is not a valid option- valid options are: " % (spark, ", ".join(os.listdir(os.environ['SPARK_CONFIG_HOME'])))
        return None
    with open(os.environ['SPARK_CONFIG_HOME'] + "/%s/spark-defaults.conf" % spark, 'r') as f:
        spark_conf = f.read()
    spark_conf_dict = {line.split( )[0] : line.split( )[1] for line in spark_conf.split("\n") if len(line.split( )) > 0}
    return spark_conf_dict

In [ ]:
test = readSparkConfig('spark21master-python3') 

In [ ]:
test['spark.files'] = '/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/full_pipeline.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/get_features.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/fcn_shannon_entro.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/mtspec.py,/gpfs/fs01/user/seff-34c2f0d3dcc620-a916a00b641d/notebook/work/runtime.py'

In [ ]:
!pwd

In [ ]:
test

In [ ]:
def writeSparkConfig(spark, spark_conf_dict):  
    """
    A function to write the updated Spark Config of DSX (overwrites previous config)
    :param spark: a string specifying which `spark-defaults.conf` to write. 
    :return: None 
    """

    if not spark in os.listdir(os.environ['SPARK_CONFIG_HOME']):
        #print "'%s' is not a valid option- valid options are: " % (spark, ", ".join(os.listdir(os.environ['SPARK_CONFIG_HOME'])))
        return None
    with open(os.environ['SPARK_CONFIG_HOME'] + "/%s/spark-defaults.conf" % spark, 'w') as f:
        for k,v in spark_conf_dict.items():
            print(k + "\t" + v + "\n")
            ln=k + "\t" + v + "\n"
            f.write(ln)
    #print "Successfully wrote new configuration to %s" % (os.environ['SPARK_CONFIG_HOME'] + "/%s/spark-defaults.conf" % spark)
    return None

In [ ]:
writeSparkConfig('spark21master-python3', test)

In [ ]:
!cp -rf $SPARK_CONFIG_HOME/spark21master-python3 $SPARK_CONFIG_HOME/spark21master-python3_bu; ls $SPARK_CONFIG_HOME

In [ ]:
!printenv